In [1]:
from pathlib import Path

import torch as th
import pytorch_lightning as pl

from lightningmodule import MULTModelWarped
from datasets import load_impressionv2_dataset_all

In [2]:
# project_name = 'shuffle'
# experiement_key = 'f7753cda73dd47399120496c09a5d5d1'
# checkpoint_dir = Path('logs')/'weights'/f'{experiement_key}_{project_name}'/f'0_{experiement_key}'/'checkpoints'
checkpoint_dir = Path('my_exp_name_find_lr/4_fa88dc4e3b0e4b8a86b8527c68314b58/checkpoints/')

In [31]:
# [train_ds, valid_ds, test_ds], target_names = load_impressionv2_dataset_all()
train_dl = th.utils.data.DataLoader(
    train_ds, batch_size=8, pin_memory=True,
)
valid_dl = th.utils.data.DataLoader(
    valid_ds, batch_size=64, pin_memory=True,
)
test_dl = th.utils.data.DataLoader(
    test_ds, batch_size=64, pin_memory=True,
)

In [4]:
ckpts = list(checkpoint_dir.glob('*.ckpt'))
assert len(ckpts) == 1
ckpt = ckpts[0]

In [5]:
ck = th.load(ckpt)

In [23]:
from argparse import Namespace
defaults = {'loss_fnc': 'L2', 'project_dim': 30, 'weight_decay': 0.0, 'optim': 'Adam'}
hyp_params = Namespace(**ck['hyper_parameters'])

In [24]:
for k, v in defaults.items():
    if k not in hyp_params:
        setattr(hyp_params, k, v)

In [32]:
trainer = pl.Trainer(gpus=1)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [33]:
model = MULTModelWarped(hyp_params, target_names=target_names, early_stopping=None)

In [34]:
model.load_state_dict(ck['state_dict'])

<All keys matched successfully>

In [35]:
trainer.test(model, test_dataloaders=test_dl)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_1mae': 0.8680583834648132,
 'test_1mae_agreeableness': 0.8784980773925781,
 'test_1mae_conscientiousness': 0.8391515016555786,
 'test_1mae_extraversion': 0.8650715947151184,
 'test_1mae_neuroticism': 0.8838691711425781,
 'test_1mae_openness': 0.8737016916275024,
 'test_loss': 0.02643299661576748}
--------------------------------------------------------------------------------


[{'test_1mae_extraversion': 0.8650715947151184,
  'test_1mae_neuroticism': 0.8838691711425781,
  'test_1mae_agreeableness': 0.8784980773925781,
  'test_1mae_conscientiousness': 0.8391515016555786,
  'test_1mae_openness': 0.8737016916275024,
  'test_loss': 0.02643299661576748,
  'test_1mae': 0.8680583834648132}]

In [36]:
trainer.test(model, test_dataloaders=valid_dl)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_1mae': 0.8915534615516663,
 'test_1mae_agreeableness': 0.9285080432891846,
 'test_1mae_conscientiousness': 0.8622080087661743,
 'test_1mae_extraversion': 0.8909584879875183,
 'test_1mae_neuroticism': 0.897257387638092,
 'test_1mae_openness': 0.8788354396820068,
 'test_loss': 0.016974149271845818}
--------------------------------------------------------------------------------


[{'test_1mae_extraversion': 0.8909584879875183,
  'test_1mae_neuroticism': 0.897257387638092,
  'test_1mae_agreeableness': 0.9285080432891846,
  'test_1mae_conscientiousness': 0.8622080087661743,
  'test_1mae_openness': 0.8788354396820068,
  'test_loss': 0.016974149271845818,
  'test_1mae': 0.8915534615516663}]